# <font color='blue'> Exercises: $S_n$, Affine cipher, and Caesar with known substring </font>

 # 1) Properties of $S_n$

 **(a) Prove that |S_n| = n!.**

*Proof (combinatorial):* A permutation of n elements is a bijection from
 the set \{1,...,n\} to itself. To build a permutation we choose an image
 for 1 in n ways, then an image for 2 in the remaining n-1 ways, and so on.
 By the rule of product there are n*(n-1)*...*1 = n! different permutations.
 Therefore |S_n| = n!.

**(b) Every permutation is a composition of transpositions.**

 *Sketch:* Any permutation can be written as a product of disjoint cycles.
 A k-cycle (a1 a2 ... ak) can be written as the product of k-1 transpositions
 (a1 ak)(a1 a{k-1})... (a1 a2). Since a permutation is a product of cycles,
 it becomes a product of transpositions.

**(c) Parity invariance:** If a permutation has two decompositions into
 transpositions with lengths m and s, then (-1)^m = (-1)^s (i.e. m and s
 have the same parity).

### Proof:  
* For a permutation $\sigma \in S_n$, define the $n \times n$ **permutation matrix** $P_\sigma$ by
$$
(P_\sigma)_{i,j} =
\begin{cases}
1 & \text{if } i = \sigma(j),\\
0 & \text{otherwise.}
\end{cases}
$$ Equivalently, the $j$-th column of $P_\sigma$ is the standard basis vector $e_{\sigma(j)}$.


* The matrix acts on basis vectors by permuting coordinates:
$$
P_\sigma e_j = e_{\sigma(j)}.
$$
If $\tau \in S_n$ then
$$
P_\sigma P_\tau e_j = P_\sigma e_{\tau(j)} = e_{\sigma(\tau(j))} = P_{\sigma\circ\tau} e_j.
$$
Since this holds for every basis vector,
$$
\boxed{P_\sigma P_\tau = P_{\sigma\circ\tau}}.
$$

* If $\tau=(a\ b)$ is a transposition, then  $\det P_{\tau}=-1$. Hence we can use formula above to prove that if  $\sigma$ is a product of m transpositions, then $\det P_{\sigma}=(-1)^m$. Since the determinant of σ is well-defined, any two transposition decompositions must give the same determinant, hence (-1)^m = (-1)^s. 
 
* Equivalently, the sign map ${\rm sgn}: S_n \to\{+1,-1\}$ is well defined and it is a group homomorphism. 

# 2) Affine cipher: encrypt and decrypt

 We use the affine cipher on the English alphabet (26 letters). Encryption
 of a letter x (0..25) is E(x) = (a*x + b) mod 26. Decryption uses the
 modular inverse a^{-1} of a modulo 26: D(y) = a^{-1} * (y - b) mod 26.

 We'll implement functions to encrypt and decrypt, then run the example
 with a = 3, b = 7 on the message: "Cryptography is wonderful"

In [2]:
from typing import Tuple

ALPHABET = 'abcdefghijklmnopqrstuvwxyz'

def modinv(a: int, m: int) -> int:
    """Return modular inverse of a mod m, or raise ValueError if none."""
    a = a % m
    # Extended Euclidean algorithm
    t0, t1 = 0, 1
    r0, r1 = m, a
    while r1 != 0:
        q = r0 // r1
        r0, r1 = r1, r0 - q * r1
        t0, t1 = t1, t0 - q * t1
    if r0 != 1:
        raise ValueError(f"{a} has no inverse modulo {m}")
    return t0 % m


def affine_encrypt(plaintext: str, a: int, b: int) -> str:
    ciphertext = []
    for ch in plaintext:
        if ch.lower() in ALPHABET:
            x = ALPHABET.index(ch.lower())
            y = (a * x + b) % 26
            c = ALPHABET[y]
            ciphertext.append(c.upper() if ch.isupper() else c)
        else:
            ciphertext.append(ch)
    return ''.join(ciphertext)


def affine_decrypt(ciphertext: str, a: int, b: int) -> str:
    a_inv = modinv(a, 26)
    plaintext = []
    for ch in ciphertext:
        if ch.lower() in ALPHABET:
            y = ALPHABET.index(ch.lower())
            x = (a_inv * (y - b)) % 26
            p = ALPHABET[x]
            plaintext.append(p.upper() if ch.isupper() else p)
        else:
            plaintext.append(ch)
    return ''.join(plaintext)

# Example
message = "Cryptography is wonderful"
a, b = 3, 7
cipher = affine_encrypt(message, a, b)
plain = affine_decrypt(cipher, a, b)

print("Original message:", message)
print("Encrypted message:", cipher)
print("Decrypted message:", plain)


Original message: Cryptography is wonderful
Encrypted message: Ngbamxzghacb fj vxuqtgwpo
Decrypted message: Cryptography is wonderful


# 3) Shift (Caesar) cipher attack 

The shift (Caesar) cipher encrypts letters as E(x) = x + k (mod 26). If we know that the plaintext contains the substring "advice" somewhere, we can try all 26 shifts and check which decrypted message contains that exact
substring. We'll implement an automatic search.

In [3]:
ct = "aopz pz tf hkcpjl: zabkf lclyf khf huk il whaplua"  # This is the cyphertext

def caesar_decrypt(ciphertext: str, k: int) -> str:
    res = []
    for ch in ciphertext:
        if ch.lower() in ALPHABET:
            i = ALPHABET.index(ch.lower())
            dec = ALPHABET[(i - k) % 26]
            res.append(dec.upper() if ch.isupper() else dec)
        else:
            res.append(ch)
    return ''.join(res)

candidates = []
for k in range(26):
    pt = caesar_decrypt(ct, k)
    if 'advice' in pt:
        candidates.append((k, pt))

print("Ciphertext:\n", ct)
print("\nCandidates (shift k, plaintext) that contain 'advice'):")
for k, pt in candidates:
    print(f"k = {k}: {pt}")

# If you run this cell you'll obtain the correct shift (the key) and the full
# decrypted message.

Ciphertext:
 aopz pz tf hkcpjl: zabkf lclyf khf huk il whaplua

Candidates (shift k, plaintext) that contain 'advice'):
k = 7: this is my advice: study every day and be patient
